# Basic CNN

In [1]:
from torch import nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.optim import SGD

def acc(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

class basic_cnn(pl.LightningModule):
    def __init__(self):
        # Define Model Here
        super(basic_cnn,self).__init__()
        #First 2d convolutional layer
        # 1 is input channel 32 is outputing feature size, 3 is kernel size
        self.conv1 = nn.Conv2d(3, 32, 3)
        # Se 2d convolutional layer, takes 32 input from previous layer
        self.conv2 = nn.Conv2d(32, 64, 3)
        
        # First fully connected layer
        self.fc1 = nn.Linear(57600,256)
        # Second fully connected layer
        self.fc2 = nn.Linear(256,128)
        # Third fully connect layer that outputs 101 labels
        self.out = nn.Linear(128,101)
        self.lr = 0.01
        self.loss = nn.CrossEntropyLoss()
        #32x57600 and 512x256
        
           
    def forward(self, x):
        # Define Forward Pass Here
        batch_size, _, _, _ = x.size()
        #print(x.size())
        
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2) 
        x = x.view(batch_size,-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.out(x)
    
    def configure_optimizers(self):
        # Define Optimizer Here
        return SGD(self.parameters(),lr = self.lr)
       
    def training_step(self, train_batch, batch_idx):
        # Define Training loop steps here
        #l_train=[]
        x, y = train_batch
        logits = self.forward(x)
        train_loss = self.loss(logits,y)
        print("Training loss = ", train_loss)
        #l_train.append(train_loss)
        self.log("train_loss", train_loss)
        return train_loss
        
    def validation_step(self, valid_batch, batch_idx):
        # Define Validation loop steps here
        #l_val=[]
        x, y = valid_batch
        logits = self.forward(x)
        val_loss = self.loss(logits,y)
        print("Validation loss = ", val_loss)
        #l_val.append(val_loss)
        
        val_pred = acc(logits, y)
        self.log("val_accuracy", val_pred[0])
        self.log("val_loss", val_loss)
        return val_loss
    
    def test_step(self, test_batch, batch_idx):
        l_test=[]
        
        x, y = test_batch
        logits = self.forward(x)
        test_loss = self.loss(logits,y)

        l_test.append(test_loss)
        
        test_pred = acc(logits, y)

        
        self.log("test_accuracy", test_pred[0])
        self.log("test_loss", test_loss)
        return l_test[-1]

In [2]:
import pytorch_lightning as pl
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
from torchvision.datasets import Food101
import torch
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization

class Food101(pl.LightningDataModule):
    def prepare_data(self):
        transform=transforms.Compose([
            transforms.RandomResizedCrop(128),
            transforms.RandomCrop(64),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5],
                                 [0.229, 0.224, 0.225])])
            #cifar10_normalization()
        
        # Data preparation
        print("[INFO] loading the FOOD 101 dataset...")
        #dataset = Food101("/C/Users/aadit/cse6363/project5", transform=transform, download=True)
        
        #dataset_size = len(dataset)
        #train_size = int(dataset_size * .95)
        #val_size = dataset_size - train_size

        #self.train_data, self.val_data = torch.utils.data.random_split(dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42))

      
        self.train_data = datasets.Food101('/C/Users/aadit/cse6363/project5', split="train", download=True, transform=transform)
        self.val_data = datasets.Food101('/C/Users/aadit/cse6363/project5', split="test", download=True, transform=transform)
        self.test_data = datasets.Food101('/C/Users/aadit/cse6363/project5', split="test", download=True, transform=transform)
        print('[INFO] data loaded successfully')

    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size= 32, shuffle=True, num_workers=8)

    def val_dataloader(self):
        return DataLoader(self.val_data, batch_size= 32, shuffle=True, num_workers=8)
    
    def test_dataloader(self):
        return DataLoader(self.test_data, batch_size= 32, shuffle=True, num_workers=8)

C:\Users\aadit\anaconda3\envs\cse6363\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Create Model Object
clf_basic_cnn = basic_cnn()
#print(clf)
# Create Data Module Object
Food101 = Food101()
# Create Trainer Object
trainer = pl.Trainer(accelerator ="cpu", max_epochs=1)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [4]:
trainer.fit(clf_basic_cnn, Food101)

[INFO] loading the FOOD 101 dataset...



  | Name  | Type             | Params
-------------------------------------------
0 | conv1 | Conv2d           | 896   
1 | conv2 | Conv2d           | 18.5 K
2 | fc1   | Linear           | 14.7 M
3 | fc2   | Linear           | 32.9 K
4 | out   | Linear           | 13.0 K
5 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
14.8 M    Trainable params
0         Non-trainable params
14.8 M    Total params
59.245    Total estimated model params size (MB)


[INFO] data loaded successfully


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\aadit\anaconda3\envs\cse6363\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:466: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(


...

Validation loss =  tensor(4.5223)
Validation loss =  tensor(4.1652)
Validation loss =  tensor(4.5119)
Validation loss =  tensor(4.4680)
Validation loss =  tensor(4.4573)
Validation loss =  tensor(4.3010)
Validation loss =  tensor(4.3332)
Validation loss =  tensor(4.4709)
Validation loss =  tensor(4.4006)
Validation loss =  tensor(4.4403)
Validation loss =  tensor(4.4260)
Validation loss =  tensor(4.6826)
Validation loss =  tensor(4.3610)
Validation loss =  tensor(4.4292)
Validation loss =  tensor(4.4429)
Validation loss =  tensor(4.3941)
Validation loss =  tensor(4.6903)
Validation loss =  tensor(4.3969)
Validation loss =  tensor(4.5612)
Validation loss =  tensor(4.4413)
Validation loss =  tensor(4.3673)
Validation loss =  tensor(4.3899)
Validation loss =  tensor(4.3790)
Validation loss =  tensor(4.4259)
Validation loss =  tensor(4.2791)
Validation loss =  tensor(4.4566)
Validation loss =  tensor(4.3962)
Validation loss =  tensor(4.5309)
Validation loss =  tensor(4.6437)
Validation los

Validation loss =  tensor(4.4127)
Validation loss =  tensor(4.3824)
Validation loss =  tensor(4.2910)
Validation loss =  tensor(4.2921)
Validation loss =  tensor(4.1454)
Validation loss =  tensor(4.2268)
Validation loss =  tensor(4.3459)
Validation loss =  tensor(4.3572)
Validation loss =  tensor(4.4536)
Validation loss =  tensor(4.0274)
Validation loss =  tensor(4.5046)
Validation loss =  tensor(4.4140)
Validation loss =  tensor(4.3602)
Validation loss =  tensor(4.4023)
Validation loss =  tensor(4.4258)
Validation loss =  tensor(4.4781)
Validation loss =  tensor(4.3099)
Validation loss =  tensor(4.1838)
Validation loss =  tensor(4.5578)
Validation loss =  tensor(4.1665)
Validation loss =  tensor(4.6172)
Validation loss =  tensor(4.3353)
Validation loss =  tensor(4.3814)
Validation loss =  tensor(4.3468)
Validation loss =  tensor(4.3760)
Validation loss =  tensor(4.5016)
Validation loss =  tensor(4.1366)
Validation loss =  tensor(4.5020)
Validation loss =  tensor(4.4158)
Validation los

Validation loss =  tensor(4.3533)
Validation loss =  tensor(4.4668)
Validation loss =  tensor(4.3141)
Validation loss =  tensor(4.5155)
Validation loss =  tensor(4.4637)
Validation loss =  tensor(4.4789)
Validation loss =  tensor(4.5064)
Validation loss =  tensor(4.4127)
Validation loss =  tensor(4.4060)
Validation loss =  tensor(4.2886)
Validation loss =  tensor(4.4134)
Validation loss =  tensor(4.4321)
Validation loss =  tensor(4.4197)
Validation loss =  tensor(4.2926)
Validation loss =  tensor(4.2250)
Validation loss =  tensor(4.4442)
Validation loss =  tensor(4.4700)
Validation loss =  tensor(4.5019)
Validation loss =  tensor(4.5308)
Validation loss =  tensor(4.3496)
Validation loss =  tensor(4.4479)
Validation loss =  tensor(4.5514)
Validation loss =  tensor(4.3981)
Validation loss =  tensor(4.2647)
Validation loss =  tensor(4.3651)
Validation loss =  tensor(4.3098)
Validation loss =  tensor(4.4251)
Validation loss =  tensor(4.4131)
Validation loss =  tensor(4.3847)
Validation los

Validation loss =  tensor(4.5917)
Validation loss =  tensor(4.4045)
Validation loss =  tensor(4.3054)
Validation loss =  tensor(4.5355)
Validation loss =  tensor(4.3323)
Validation loss =  tensor(4.4728)
Validation loss =  tensor(4.4966)
Validation loss =  tensor(4.4659)
Validation loss =  tensor(4.4883)
Validation loss =  tensor(4.5459)
Validation loss =  tensor(4.5611)
Validation loss =  tensor(4.4968)
Validation loss =  tensor(4.4296)
Validation loss =  tensor(4.3025)
Validation loss =  tensor(4.3858)
Validation loss =  tensor(4.4317)
Validation loss =  tensor(4.1772)
Validation loss =  tensor(4.4014)
Validation loss =  tensor(4.7007)
Validation loss =  tensor(4.5002)
Validation loss =  tensor(4.3518)
Validation loss =  tensor(4.5495)
Validation loss =  tensor(4.4155)
Validation loss =  tensor(4.5373)
Validation loss =  tensor(4.3037)
Validation loss =  tensor(4.4944)
Validation loss =  tensor(4.1766)
Validation loss =  tensor(4.2407)
Validation loss =  tensor(4.4234)
Validation los

`Trainer.fit` stopped: `max_epochs=1` reached.


In [7]:
trainer.test(clf_basic_cnn, Food101)

[INFO] loading the FOOD 101 dataset...
[INFO] data loaded successfully


C:\Users\aadit\anaconda3\envs\cse6363\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:466: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy          3.869307041168213
        test_loss            4.420573711395264
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_accuracy': 3.869307041168213, 'test_loss': 4.420573711395264}]

# All Convolutional Net

In [14]:


def acc(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

class all_conv_net(pl.LightningModule):
    def __init__(self):
        # Define Model Here
        super(all_conv_net,self).__init__()
        #First 2d convolutional layer
        # 1 is input channel 32 is outputing feature size, 3 is kernel size
        self.conv1 = nn.Conv2d(3, 32, 3)
        # Se 2d convolutional layer, takes 32 input from previous layer
        self.conv2 = nn.Conv2d(32, 64, 3)
        
        self.conv3 = nn.Conv2d(64, 101, 3)
        # First fully connected layer
        #self.fc1 = nn.Linear(57600,256)
        # Second fully connected layer
        #self.fc2 = nn.Linear(256,128)
        # Third fully connect layer that outputs 101 labels
        #self.out = nn.Linear(128,101)
        self.lr = 0.01
        self.loss = nn.CrossEntropyLoss()
        #32x57600 and 512x256
        
           
    def forward(self, x):
        # Define Forward Pass Here
        batch_size, _, _, _ = x.size()
        #print(x.size())
        
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2) 
        #x = x.view(batch_size,-1)
        #x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = x.flatten(1)
        return x
    
    def configure_optimizers(self):
        # Define Optimizer Here
        return SGD(self.parameters(),lr = self.lr)
       
    def training_step(self, train_batch, batch_idx):
        # Define Training loop steps here
        #l_train=[]
        x, y = train_batch
        logits = self.forward(x)
        train_loss = self.loss(logits,y)
        print("Training loss = ", train_loss)
        #l_train.append(train_loss)
        self.log("train_loss", train_loss)
        return train_loss
        
    def validation_step(self, valid_batch, batch_idx):
        # Define Validation loop steps here
        #l_val=[]
        x, y = valid_batch
        logits = self.forward(x)
        val_loss = self.loss(logits,y)
        print("Validation loss = ", val_loss)
        #l_val.append(val_loss)
        
        val_pred = acc(logits, y)
        self.log("val_accuracy", val_pred[0])
        self.log("val_loss", val_loss)
        return val_loss
    
    def test_step(self, test_batch, batch_idx):
        l_test=[]
        
        x, y = test_batch
        logits = self.forward(x)
        test_loss = self.loss(logits,y)

        l_test.append(test_loss)
        
        test_pred = acc(logits, y)

        
        self.log("test_accuracy", test_pred[0])
        self.log("test_loss", test_loss)
        return l_test[-1]

In [15]:
# Create Model Object
clf_all_conv_net = all_conv_net()
#print(clf)
# Create Data Module Object
#Food101 = Food101()
# Create Trainer Object
trainer = pl.Trainer(accelerator ="cpu", max_epochs=1)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [16]:
trainer.fit(clf_all_conv_net, Food101)

[INFO] loading the FOOD 101 dataset...



  | Name  | Type             | Params
-------------------------------------------
0 | conv1 | Conv2d           | 896   
1 | conv2 | Conv2d           | 18.5 K
2 | conv3 | Conv2d           | 58.3 K
3 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
77.7 K    Trainable params
0         Non-trainable params
77.7 K    Total params
0.311     Total estimated model params size (MB)


[INFO] data loaded successfully


Sanity Checking: 0it [00:00, ?it/s]

Validation loss =  tensor(11.3813)
Validation loss =  tensor(11.3861)


...

Validation loss =  tensor(6.7652)
Validation loss =  tensor(6.7114)
Validation loss =  tensor(6.7918)
Validation loss =  tensor(6.7317)
Validation loss =  tensor(6.7306)
Validation loss =  tensor(6.7444)
Validation loss =  tensor(6.7201)
Validation loss =  tensor(6.7878)
Validation loss =  tensor(6.6917)
Validation loss =  tensor(6.8489)
Validation loss =  tensor(6.6816)
Validation loss =  tensor(6.6881)
Validation loss =  tensor(6.7620)
Validation loss =  tensor(6.7763)
Validation loss =  tensor(6.7143)
Validation loss =  tensor(6.7749)
Validation loss =  tensor(6.6968)
Validation loss =  tensor(6.7023)
Validation loss =  tensor(6.7091)
Validation loss =  tensor(6.7799)
Validation loss =  tensor(6.6983)
Validation loss =  tensor(6.7391)
Validation loss =  tensor(6.7868)
Validation loss =  tensor(6.7480)
Validation loss =  tensor(6.7783)
Validation loss =  tensor(6.6952)
Validation loss =  tensor(6.7444)
Validation loss =  tensor(6.7331)
Validation loss =  tensor(6.6855)
Validation los

Validation loss =  tensor(6.7612)
Validation loss =  tensor(6.7432)
Validation loss =  tensor(6.7387)
Validation loss =  tensor(6.6443)
Validation loss =  tensor(6.7389)
Validation loss =  tensor(6.7442)
Validation loss =  tensor(6.7808)
Validation loss =  tensor(6.7997)
Validation loss =  tensor(6.6915)
Validation loss =  tensor(6.7390)
Validation loss =  tensor(6.7106)
Validation loss =  tensor(6.6908)
Validation loss =  tensor(6.7883)
Validation loss =  tensor(6.7459)
Validation loss =  tensor(6.8091)
Validation loss =  tensor(6.7335)
Validation loss =  tensor(6.7539)
Validation loss =  tensor(6.7851)
Validation loss =  tensor(6.7222)
Validation loss =  tensor(6.8124)
Validation loss =  tensor(6.7693)
Validation loss =  tensor(6.7359)
Validation loss =  tensor(6.7439)
Validation loss =  tensor(6.7078)
Validation loss =  tensor(6.7176)
Validation loss =  tensor(6.7253)
Validation loss =  tensor(6.7699)
Validation loss =  tensor(6.7321)
Validation loss =  tensor(6.7577)
Validation los

Validation loss =  tensor(6.7666)
Validation loss =  tensor(6.7877)
Validation loss =  tensor(6.7353)
Validation loss =  tensor(6.7337)
Validation loss =  tensor(6.7645)
Validation loss =  tensor(6.7301)
Validation loss =  tensor(6.7362)
Validation loss =  tensor(6.7408)
Validation loss =  tensor(6.7582)
Validation loss =  tensor(6.7053)
Validation loss =  tensor(6.6439)
Validation loss =  tensor(6.7891)
Validation loss =  tensor(6.7653)
Validation loss =  tensor(6.6974)
Validation loss =  tensor(6.7117)
Validation loss =  tensor(6.7414)
Validation loss =  tensor(6.6703)
Validation loss =  tensor(6.7104)
Validation loss =  tensor(6.7138)
Validation loss =  tensor(6.7765)
Validation loss =  tensor(6.7683)
Validation loss =  tensor(6.7035)
Validation loss =  tensor(6.7146)
Validation loss =  tensor(6.7594)
Validation loss =  tensor(6.7157)
Validation loss =  tensor(6.7035)
Validation loss =  tensor(6.7775)
Validation loss =  tensor(6.7220)
Validation loss =  tensor(6.7975)
Validation los

`Trainer.fit` stopped: `max_epochs=1` reached.


tensor(6.7891)
Validation loss =  tensor(6.6397)


In [17]:
trainer.test(clf_all_conv_net, Food101)

[INFO] loading the FOOD 101 dataset...
[INFO] data loaded successfully


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.23366336524486542
        test_loss            6.738712787628174
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_accuracy': 0.23366336524486542, 'test_loss': 6.738712787628174}]

# Regularization

In [18]:
def acc(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

class reg(pl.LightningModule):
    def __init__(self):
        # Define Model Here
        super(reg,self).__init__()
        #First 2d convolutional layer
        # 1 is input channel 32 is outputing feature size, 3 is kernel size
        self.conv1 = nn.Conv2d(3, 32, 3)
        # Se 2d convolutional layer, takes 32 input from previous layer
        self.conv2 = nn.Conv2d(32, 64, 3)
        # First fully connected layer
        self.fc1 = nn.Linear(57600,256)
        # Second fully connected layer
        self.fc2 = nn.Linear(256,128)
        # Third fully connect layer that outputs 101 labels
        self.out = nn.Linear(128,101)
        
        #Define proportion or neurons to dropout
        self.dropout = nn.Dropout(0.25)

        
        self.lr = 0.01
        self.loss = nn.CrossEntropyLoss()
        #32x57600 and 512x256
        
           
    def forward(self, x):
        # Define Forward Pass Here
        batch_size, _, _, _ = x.size()
        #print(x.size())
        
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2) 
        x = x.view(batch_size,-1)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        return self.out(x)
    
    def configure_optimizers(self):
        # Define Optimizer Here
        return SGD(self.parameters(),lr = self.lr)
       
    def training_step(self, train_batch, batch_idx):
        # Define Training loop steps here
        #l_train=[]
        x, y = train_batch
        logits = self.forward(x)
        train_loss = self.loss(logits,y)
        print("Training loss = ", train_loss)
        #l_train.append(train_loss)
        self.log("train_loss", train_loss)
        return train_loss
        
    def validation_step(self, valid_batch, batch_idx):
        # Define Validation loop steps here
        #l_val=[]
        x, y = valid_batch
        logits = self.forward(x)
        val_loss = self.loss(logits,y)
        print("Validation loss = ", val_loss)
        #l_val.append(val_loss)
        
        val_pred = acc(logits, y)
        self.log("val_accuracy", val_pred[0])
        self.log("val_loss", val_loss)
        return val_loss
    
    def test_step(self, test_batch, batch_idx):
        l_test=[]
        
        x, y = test_batch
        logits = self.forward(x)
        test_loss = self.loss(logits,y)

        l_test.append(test_loss)
        
        test_pred = acc(logits, y)

        
        self.log("test_accuracy", test_pred[0])
        self.log("test_loss", test_loss)
        return l_test[-1]

In [19]:
import pytorch_lightning as pl
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
from torchvision.datasets import Food101
import torch
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization

class aug_Food101(pl.LightningDataModule):
    def prepare_data(self):
        transform=transforms.Compose([
            transforms.RandomResizedCrop(128),
            transforms.RandAugment(),
            transforms.RandomCrop(64),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5],
                                 [0.229, 0.224, 0.225])])
            #cifar10_normalization()
        
        # Data preparation
        print("[INFO] loading the FOOD 101 dataset...")
        #dataset = Food101("/C/Users/aadit/cse6363/project5", transform=transform, download=True)
        
        #dataset_size = len(dataset)
        #train_size = int(dataset_size * .95)
        #val_size = dataset_size - train_size

        #self.train_data, self.val_data = torch.utils.data.random_split(dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42))

      
        self.train_data = datasets.Food101('/C/Users/aadit/cse6363/project5', split="train", download=True, transform=transform)
        self.val_data = datasets.Food101('/C/Users/aadit/cse6363/project5', split="test", download=True, transform=transform)
        self.test_data = datasets.Food101('/C/Users/aadit/cse6363/project5', split="test", download=True, transform=transform)
        print('[INFO] data loaded successfully')

    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size= 32, shuffle=True, num_workers=8)

    def val_dataloader(self):
        return DataLoader(self.val_data, batch_size= 32, shuffle=True, num_workers=8)
    
    def test_dataloader(self):
        return DataLoader(self.test_data, batch_size= 32, shuffle=True, num_workers=8)

In [20]:
# Create Model Object
clf_reg = reg()
#print(clf)
# Create Data Module Object
aug_Food101 = aug_Food101()
# Create Trainer Object
trainer = pl.Trainer(accelerator ="cpu", max_epochs=1)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [21]:
trainer.fit(clf_reg, aug_Food101)

[INFO] loading the FOOD 101 dataset...



  | Name    | Type             | Params
---------------------------------------------
0 | conv1   | Conv2d           | 896   
1 | conv2   | Conv2d           | 18.5 K
2 | fc1     | Linear           | 14.7 M
3 | fc2     | Linear           | 32.9 K
4 | out     | Linear           | 13.0 K
5 | dropout | Dropout          | 0     
6 | loss    | CrossEntropyLoss | 0     
---------------------------------------------
14.8 M    Trainable params
0         Non-trainable params
14.8 M    Total params
59.245    Total estimated model params size (MB)


[INFO] data loaded successfully


Sanity Checking: 0it [00:00, ?it/s]

Validation loss =  tensor(4.5947)
Validation loss =  tensor(4.6068)


...

Validation loss =  tensor(4.4698)
Validation loss =  tensor(4.5343)
Validation loss =  tensor(4.3952)
Validation loss =  tensor(4.5492)
Validation loss =  tensor(4.5333)
Validation loss =  tensor(4.5637)
Validation loss =  tensor(4.4935)
Validation loss =  tensor(4.4248)
Validation loss =  tensor(4.2980)
Validation loss =  tensor(4.4404)
Validation loss =  tensor(4.5441)
Validation loss =  tensor(4.4383)
Validation loss =  tensor(4.6054)
Validation loss =  tensor(4.5889)
Validation loss =  tensor(4.4450)
Validation loss =  tensor(4.5689)
Validation loss =  tensor(4.4913)
Validation loss =  tensor(4.5361)
Validation loss =  tensor(4.4200)
Validation loss =  tensor(4.5853)
Validation loss =  tensor(4.3614)
Validation loss =  tensor(4.6444)
Validation loss =  tensor(4.3121)
Validation loss =  tensor(4.5838)
Validation loss =  tensor(4.4329)
Validation loss =  tensor(4.5655)
Validation loss =  tensor(4.4969)
Validation loss =  tensor(4.5702)
Validation loss =  tensor(4.4721)
Validation los

Validation loss =  tensor(4.4458)
Validation loss =  tensor(4.5037)
Validation loss =  tensor(4.3869)
Validation loss =  tensor(4.5485)
Validation loss =  tensor(4.5352)
Validation loss =  tensor(4.5426)
Validation loss =  tensor(4.5451)
Validation loss =  tensor(4.4904)
Validation loss =  tensor(4.5768)
Validation loss =  tensor(4.6209)
Validation loss =  tensor(4.5372)
Validation loss =  tensor(4.5185)
Validation loss =  tensor(4.5640)
Validation loss =  tensor(4.4285)
Validation loss =  tensor(4.6765)
Validation loss =  tensor(4.5747)
Validation loss =  tensor(4.4637)
Validation loss =  tensor(4.5648)
Validation loss =  tensor(4.5544)
Validation loss =  tensor(4.4242)
Validation loss =  tensor(4.4926)
Validation loss =  tensor(4.5093)
Validation loss =  tensor(4.5448)
Validation loss =  tensor(4.4797)
Validation loss =  tensor(4.5068)
Validation loss =  tensor(4.4414)
Validation loss =  tensor(4.4943)
Validation loss =  tensor(4.5494)
Validation loss =  tensor(4.5411)
Validation los

Validation loss =  tensor(4.4992)
Validation loss =  tensor(4.4987)
Validation loss =  tensor(4.4601)
Validation loss =  tensor(4.5143)
Validation loss =  tensor(4.4327)
Validation loss =  tensor(4.5290)
Validation loss =  tensor(4.3285)
Validation loss =  tensor(4.5922)
Validation loss =  tensor(4.3238)
Validation loss =  tensor(4.5043)
Validation loss =  tensor(4.5025)
Validation loss =  tensor(4.2638)
Validation loss =  tensor(4.4190)
Validation loss =  tensor(4.4209)
Validation loss =  tensor(4.6123)
Validation loss =  tensor(4.3917)
Validation loss =  tensor(4.5263)
Validation loss =  tensor(4.6999)
Validation loss =  tensor(4.4819)
Validation loss =  tensor(4.5361)
Validation loss =  tensor(4.3838)
Validation loss =  tensor(4.4998)
Validation loss =  tensor(4.3457)
Validation loss =  tensor(4.4895)
Validation loss =  tensor(4.6097)
Validation loss =  tensor(4.4955)
Validation loss =  tensor(4.5111)
Validation loss =  tensor(4.4365)
Validation loss =  tensor(4.5835)
Validation los

Validation loss =  tensor(4.5045)
Validation loss =  tensor(4.4633)
Validation loss =  tensor(4.5343)
Validation loss =  tensor(4.5700)
Validation loss =  tensor(4.4579)
Validation loss =  tensor(4.3945)
Validation loss =  tensor(4.5475)
Validation loss =  tensor(4.4352)
Validation loss =  tensor(4.4827)
Validation loss =  tensor(4.5693)
Validation loss =  tensor(4.4563)
Validation loss =  tensor(4.5193)
Validation loss =  tensor(4.5414)
Validation loss =  tensor(4.4411)
Validation loss =  tensor(4.4154)
Validation loss =  tensor(4.3682)
Validation loss =  tensor(4.6146)
Validation loss =  tensor(4.4567)
Validation loss =  tensor(4.5252)
Validation loss =  tensor(4.5685)
Validation loss =  tensor(4.4182)
Validation loss =  tensor(4.4792)
Validation loss =  tensor(4.5227)
Validation loss =  tensor(4.5088)
Validation loss =  tensor(4.5203)
Validation loss =  tensor(4.3995)
Validation loss =  tensor(4.4466)
Validation loss =  tensor(4.5056)
Validation loss =  tensor(4.5499)
Validation los

`Trainer.fit` stopped: `max_epochs=1` reached.


In [22]:
trainer.test(clf_reg, aug_Food101)

[INFO] loading the FOOD 101 dataset...
[INFO] data loaded successfully


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         2.8039603233337402
        test_loss            4.498032569885254
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_accuracy': 2.8039603233337402, 'test_loss': 4.498032569885254}]

# Transfer Learning

In [61]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import transforms
from torchvision.datasets import Food101
from pytorch_lightning.callbacks import ModelCheckpoint
import pytorch_lightning as pl

def acc(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

class ImagenetTransferLearning(pl.LightningModule):
    def __init__(self, data_path, batch_size, lr):
        super().__init__()
        
        self.data_path = data_path
        self.batch_size = batch_size
        self.lr = lr
        
        # Data preparation
        dataset = Food101(data_path, transform=transforms.Compose([
            transforms.RandAugment(),
            transforms.RandomCrop(56),
            transforms.ToTensor(),
        ]), download=True)

        dataset_size = len(dataset)
        train_size = int(dataset_size * .95)
        val_size = dataset_size - train_size

        self.train_dataset, self.val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

        # Loss function
        self.loss_fn = nn.CrossEntropyLoss()
        
        # init a pretrained resnet
        backbone = models.resnet18(pretrained=True)
        num_filters = backbone.fc.in_features
        layers = list(backbone.children())[:-1]
        self.feature_extractor = nn.Sequential(*layers)

        # use the pretrained model to classify food101
        num_target_classes = 101
        self.classifier = nn.Linear(num_filters, num_target_classes)

    def forward(self, x):
        self.feature_extractor.eval()
        with torch.no_grad():
            representations = self.feature_extractor(x).flatten(1)
        x = self.classifier(representations)
        
        return x
        
    def training_step(self, batch, batch_idx):
        input, target = batch
        output = self(input)
        loss = self.loss_fn(output, target)
        
        self.log("train_loss", loss)
        
        return loss
        
    def validation_step(self, batch, batch_idx):
        input, target = batch
        output = self(input)
        loss = self.loss_fn(output, target)
        
        self.log("val_loss", loss)
        
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.lr)
        
    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_dataset,
                                           batch_size=self.batch_size,
                                           num_workers=8,
                                           shuffle=True)
    
    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_dataset,
                                           batch_size=self.batch_size,
                                           num_workers=8,
                                           shuffle=False)
    
    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.val_dataset,
                                           batch_size=self.batch_size,
                                           num_workers=8,
                                           shuffle=False)
    
    def test_step(self, test_batch, batch_idx):
        l_test=[]
        
        x, y = test_batch
        logits = self.forward(x)
        test_loss = self.loss_fn(logits,y)

        l_test.append(test_loss)
        
        test_pred = acc(logits, y)

        
        self.log("test_accuracy", test_pred[0])
        self.log("test_loss", test_loss)
        return l_test[-1]

In [62]:
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min")

In [63]:
model = ImagenetTransferLearning("/C/Users/aadit/cse6363/project5", 32, 1e-3)

In [64]:
trainer = pl.Trainer(accelerator="cpu", callbacks=[checkpoint_callback], max_epochs=1)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [65]:
trainer.fit(model)


  | Name              | Type             | Params
-------------------------------------------------------
0 | loss_fn           | CrossEntropyLoss | 0     
1 | feature_extractor | Sequential       | 11.2 M
2 | classifier        | Linear           | 51.8 K
-------------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.913    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [66]:
trainer.test(model)

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy          4.963041305541992
        test_loss            4.55490255355835
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_accuracy': 4.963041305541992, 'test_loss': 4.55490255355835}]

# Bonus Round

In [70]:
def acc(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

class bonus(pl.LightningModule):
    def __init__(self):
        # Define Model Here
        super(bonus,self).__init__()
        #First 2d convolutional layer
        # 1 is input channel 32 is outputing feature size, 3 is kernel size
        #self.conv1 = nn.Conv2d(3, 32, 3)
        # Se 2d convolutional layer, takes 32 input from previous layer
        #self.conv2 = nn.Conv2d(32, 64, 3)
        
        self.conv1 = nn.Conv2d(3, 101, 3)
        # First fully connected layer
        #self.fc1 = nn.Linear(57600,256)
        # Second fully connected layer
        #self.fc2 = nn.Linear(256,128)
        # Third fully connect layer that outputs 101 labels
        #self.out = nn.Linear(128,101)
        self.lr = 0.01
        self.loss = nn.CrossEntropyLoss()
        #32x57600 and 512x256
        
           
    def forward(self, x):
        # Define Forward Pass Here
        batch_size, _, _, _ = x.size()
        #print(x.size())
        
        x = F.relu(self.conv1(x))
        #x = F.relu(self.conv2(x))
        #x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2) 
        #x = x.view(batch_size,-1)
        #x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = x.flatten(1)
        return x
    
    def configure_optimizers(self):
        # Define Optimizer Here
        return SGD(self.parameters(),lr = self.lr)
       
    def training_step(self, train_batch, batch_idx):
        # Define Training loop steps here
        #l_train=[]
        x, y = train_batch
        logits = self.forward(x)
        train_loss = self.loss(logits,y)
        print("Training loss = ", train_loss)
        #l_train.append(train_loss)
        self.log("train_loss", train_loss)
        return train_loss
        
    def validation_step(self, valid_batch, batch_idx):
        # Define Validation loop steps here
        #l_val=[]
        x, y = valid_batch
        logits = self.forward(x)
        val_loss = self.loss(logits,y)
        print("Validation loss = ", val_loss)
        #l_val.append(val_loss)
        
        val_pred = acc(logits, y)
        self.log("val_accuracy", val_pred[0])
        self.log("val_loss", val_loss)
        return val_loss
    
    def test_step(self, test_batch, batch_idx):
        l_test=[]
        
        x, y = test_batch
        logits = self.forward(x)
        test_loss = self.loss(logits,y)

        l_test.append(test_loss)
        
        test_pred = acc(logits, y)

        
        self.log("test_accuracy", test_pred[0])
        self.log("test_loss", test_loss)
        return l_test[-1]

In [71]:
class Food101_bonus(pl.LightningDataModule):
    def prepare_data(self):
        transform=transforms.Compose([
            transforms.RandomResizedCrop(128),
            transforms.RandomCrop(56),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5],
                                 [0.229, 0.224, 0.225])])
            #cifar10_normalization()
        
        # Data preparation
        print("[INFO] loading the FOOD 101 dataset...")
        #dataset = Food101("/C/Users/aadit/cse6363/project5", transform=transform, download=True)
        
        #dataset_size = len(dataset)
        #train_size = int(dataset_size * .95)
        #val_size = dataset_size - train_size

        #self.train_data, self.val_data = torch.utils.data.random_split(dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42))

      
        self.train_data = datasets.Food101('/C/Users/aadit/cse6363/project5', split="train", download=True, transform=transform)
        self.val_data = datasets.Food101('/C/Users/aadit/cse6363/project5', split="test", download=True, transform=transform)
        self.test_data = datasets.Food101('/C/Users/aadit/cse6363/project5', split="test", download=True, transform=transform)
        print('[INFO] data loaded successfully')

    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size= 32, shuffle=True, num_workers=8)

    def val_dataloader(self):
        return DataLoader(self.val_data, batch_size= 32, shuffle=True, num_workers=8)
    
    def test_dataloader(self):
        return DataLoader(self.test_data, batch_size= 32, shuffle=True, num_workers=8)

In [72]:
# Create Model Object
clf_bonus = bonus()
#print(clf)
# Create Data Module Object
Food101_bonus = Food101_bonus()
# Create Trainer Object
trainer = pl.Trainer(accelerator ="cpu", max_epochs=1)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [73]:
trainer.fit(clf_bonus, Food101_bonus)

[INFO] loading the FOOD 101 dataset...



  | Name  | Type             | Params
-------------------------------------------
0 | conv1 | Conv2d           | 2.8 K 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
2.8 K     Trainable params
0         Non-trainable params
2.8 K     Total params
0.011     Total estimated model params size (MB)


[INFO] data loaded successfully


Sanity Checking: 0it [00:00, ?it/s]

Validation loss =  tensor(11.3669)
Validation loss =  tensor(11.2644)


...

Validation loss =  tensor(6.6908)
Validation loss =  tensor(6.6793)
Validation loss =  tensor(6.7372)
Validation loss =  tensor(6.6799)
Validation loss =  tensor(6.7455)
Validation loss =  tensor(6.7065)
Validation loss =  tensor(6.7160)
Validation loss =  tensor(6.7355)
Validation loss =  tensor(6.7833)
Validation loss =  tensor(6.8058)
Validation loss =  tensor(6.7503)
Validation loss =  tensor(6.7034)
Validation loss =  tensor(6.7393)
Validation loss =  tensor(6.7480)
Validation loss =  tensor(6.6895)
Validation loss =  tensor(6.8016)
Validation loss =  tensor(6.8645)
Validation loss =  tensor(6.6673)
Validation loss =  tensor(6.7850)
Validation loss =  tensor(6.7066)
Validation loss =  tensor(6.7023)
Validation loss =  tensor(6.6741)
Validation loss =  tensor(6.7448)
Validation loss =  tensor(6.6619)
Validation loss =  tensor(6.8248)
Validation loss =  tensor(6.7135)
Validation loss =  tensor(6.7416)
Validation loss =  tensor(6.8517)
Validation loss =  tensor(6.7447)
Validation los

Validation loss =  tensor(6.6969)
Validation loss =  tensor(6.7495)
Validation loss =  tensor(6.7329)
Validation loss =  tensor(6.7877)
Validation loss =  tensor(6.7486)
Validation loss =  tensor(6.7163)
Validation loss =  tensor(6.8036)
Validation loss =  tensor(6.7936)
Validation loss =  tensor(6.6495)
Validation loss =  tensor(6.7624)
Validation loss =  tensor(6.7825)
Validation loss =  tensor(6.7094)
Validation loss =  tensor(6.8128)
Validation loss =  tensor(6.8311)
Validation loss =  tensor(6.7586)
Validation loss =  tensor(6.7420)
Validation loss =  tensor(6.7384)
Validation loss =  tensor(6.7577)
Validation loss =  tensor(6.8218)
Validation loss =  tensor(6.7957)
Validation loss =  tensor(6.7808)
Validation loss =  tensor(6.7974)
Validation loss =  tensor(6.6814)
Validation loss =  tensor(6.8128)
Validation loss =  tensor(6.7298)
Validation loss =  tensor(6.7411)
Validation loss =  tensor(6.6584)
Validation loss =  tensor(6.7765)
Validation loss =  tensor(6.7866)
Validation los

Validation loss =  tensor(6.7268)
Validation loss =  tensor(6.6903)
Validation loss =  tensor(6.7431)
Validation loss =  tensor(6.7670)
Validation loss =  tensor(6.7387)
Validation loss =  tensor(6.8019)
Validation loss =  tensor(6.7537)
Validation loss =  tensor(6.7816)
Validation loss =  tensor(6.6590)
Validation loss =  tensor(6.7298)
Validation loss =  tensor(6.7657)
Validation loss =  tensor(6.7478)
Validation loss =  tensor(6.7589)
Validation loss =  tensor(6.7130)
Validation loss =  tensor(6.7609)
Validation loss =  tensor(6.7529)
Validation loss =  tensor(6.7588)
Validation loss =  tensor(6.7723)
Validation loss =  tensor(6.7770)
Validation loss =  tensor(6.8689)
Validation loss =  tensor(6.7577)
Validation loss =  tensor(6.7474)
Validation loss =  tensor(6.7698)
Validation loss =  tensor(6.6896)
Validation loss =  tensor(6.7591)
Validation loss =  tensor(6.7323)
Validation loss =  tensor(6.7580)
Validation loss =  tensor(6.6959)
Validation loss =  tensor(6.7478)
Validation los

`Trainer.fit` stopped: `max_epochs=1` reached.


Validation loss =  tensor(6.7482)
Validation loss =  tensor(6.5264)


In [74]:
trainer.test(clf_bonus, Food101_bonus)

[INFO] loading the FOOD 101 dataset...
[INFO] data loaded successfully


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.22574257850646973
        test_loss            6.749154567718506
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_accuracy': 0.22574257850646973, 'test_loss': 6.749154567718506}]